In [2]:
import autograd.numpy as np
from autograd import grad, jacobian
import scipy.linalg as sp_linalg

In [3]:
# Define constants

# Motor
Rm = 8.4  # Resistance
kt = 0.042  # Current-torque (N-m/A)
km = 0.042  # Back-emf constant (V-s/rad)

# Rotary Arm
mr = 0.095  # Mass (kg)
Lr = 0.085  # Total length (m)
Jr = mr * Lr ** 2 / 12  # Moment of inertia about pivot (kg-m^2)
Br = Dr = 0.0015  # Equivalent viscous damping coefficient (N-m-s/rad)

# Pendulum Link
mp = 0.024  # Mass (kg)
Lp = 0.129  # Total length (m)
Jp = mp * Lp ** 2 / 12  # Moment of inertia about pivot (kg-m^2)
Dp = Bp = 0.0005  # Equivalent viscous damping coefficient (N-m-s/rad)

g = 9.81  # Gravity constant

In [4]:
def forward_model(state, action, dt=1/250):
    theta = state[0]
    alpha = state[1]
    theta_dot = state[2]
    alpha_dot = state[3]
    Vm = action

    tau = (km * (Vm - km * theta_dot)) / Rm  # torque

    theta_dot_dot = (-Lp*Lr*mp*(-8.0*Dp*alpha_dot + Lp**2*mp*theta_dot**2*np.sin(2.0*alpha) + 4.0*Lp*g*mp*np.sin(alpha))*np.cos(alpha) + (4.0*Jp + Lp**2*mp)*(4.0*Dr*theta_dot + Lp**2*mp*alpha_dot*theta_dot*np.sin(2.0*alpha) + 2.0*Lp*Lr*mp*alpha_dot**2*np.sin(alpha) - 4.0*tau))/(4.0*Lp**2*Lr**2*mp**2*np.cos(alpha)**2 - (4.0*Jp + Lp**2*mp)*(4.0*Jr + Lp**2*mp*np.sin(alpha)**2 + 4.0*Lr**2*mp))
    alpha_dot_dot = (2.0*Lp*Lr*mp*(4.0*Dr*theta_dot + Lp**2*mp*alpha_dot*theta_dot*np.sin(2.0*alpha) + 2.0*Lp*Lr*mp*alpha_dot**2*np.sin(alpha) - 4.0*tau)*np.cos(alpha) - 0.5*(4.0*Jr + Lp**2*mp*np.sin(alpha)**2 + 4.0*Lr**2*mp)*(-8.0*Dp*alpha_dot + Lp**2*mp*theta_dot**2*np.sin(2.0*alpha) + 4.0*Lp*g*mp*np.sin(alpha)))/(4.0*Lp**2*Lr**2*mp**2*np.cos(alpha)**2 - (4.0*Jp + Lp**2*mp)*(4.0*Jr + Lp**2*mp*np.sin(alpha)**2 + 4.0*Lr**2*mp))

    # Works around a single operating point
    theta += theta_dot * dt
    alpha += alpha_dot * dt
    theta_dot += theta_dot_dot * dt
    alpha_dot += alpha_dot_dot * dt

    theta %= 2 * np.pi
    alpha %= 2 * np.pi

    # For continuous version of LQR
    state = np.array([theta_dot, alpha_dot, theta_dot_dot, alpha_dot_dot])

    # For discrete version of LQR
#     state = np.array([theta, alpha, theta_dot, alpha_dot])
    return state


In [5]:
def computeAB(forward_dynamics_model, current_state, current_control):
    # Linearizing Dynamics
    a_mat = jacobian(forward_dynamics_model, 0)
    b_mat = jacobian(forward_dynamics_model, 1)
    A = a_mat(current_state, current_control)
    B = b_mat(current_state, current_control)

    # Correct continuous time linearization from Quanser Workbook -
    # A = np.array(
    #     [
    #         [0,0,1.0000,0],
    #         [0,0,0 ,1.0000],
    #         [0,149.2751,-0.0104,0],
    #         [0,261.6091,-0.0103,0]
    #     ])
    # B = np.array([ 0,0,49.7275,49.1493])

    return A, B

In [6]:
def LQR_control(A, B, ):
    # Cost matrices for LQR
    Q = np.diag(np.array([1, 1, 1, 1]))  # state_dimension = 4
    R = np.eye(1)  # control_dimension = 1

    # Use if discrete forward dynamics is used
#     X = sp_linalg.solve_discrete_are(A, B, Q, R)
#     K = np.dot(np.linalg.pinv(R + np.dot(B.T, np.dot(X, B))), np.dot(B.T, np.dot(X, A)))

    # Use for continuous version of LQR
    X = sp_linalg.solve_continuous_are(A, B, Q, R)
    K = np.dot(np.linalg.pinv(R), np.dot(B.T, X))
    return np.squeeze(K, 0)

In [7]:
# Where you want to linearize the system around
linearization_state = np.zeros((4,))
linearization_action = np.array([0]) # Just a vector with 0


In [8]:
A, B = computeAB(forward_model, linearization_state, linearization_action)
print(f"A = \n{A}\nB = {B}")

print("""
Compare to:
A = 
[[0, 0,        1,       0],
 [0, 0,        0,       1],
 [0, 149.2751, -0.0104, 0],
 [0, 261.6091, -0.0103, 0]]
B = [ 0,0,49.7275,49.1493]
""")

A = 
[[[ 0.00000000e+00  5.97100387e-01  9.31972733e-01 -1.96597230e-02]]

 [[ 0.00000000e+00  1.04643643e+00 -6.72362525e-02  9.65545743e-01]]

 [[ 0.00000000e+00  1.49275097e+02 -1.70068168e+01 -4.91493074e+00]]

 [[ 0.00000000e+00  2.61609107e+02 -1.68090631e+01 -8.61356429e+00]]]
B = [[[ 0.19891014]]

 [[ 0.19659723]]

 [[49.72753455]]

 [[49.1493074 ]]]

Compare to:
A = 
[[0, 0,        1,       0],
 [0, 0,        0,       1],
 [0, 149.2751, -0.0104, 0],
 [0, 261.6091, -0.0103, 0]]
B = [ 0,0,49.7275,49.1493]



In [9]:
A = A.reshape(4,4)
B = B.reshape(4,1)


In [75]:
k = LQR_control(A, B)

print(f"k = {k}")
print("Compared to Quanser's:\nk = [-2.0, 35.0, -1.5, 3.0]")

k = [-1.         40.10113264 -1.64935946  2.92022676]
Compared to Quanser's:
k = [-2.0, 35.0, -1.5, 3.0]


In [80]:
# k = [-0.86938544 36.92064594 -1.49277614  2.74003595]
# Compared to Quanser's:
# k = [-2.0, 35.0, -1.5, 3.0]

In [84]:
# Quanser's equations
Jt = Jr*Jp + mp*(Lp/2)**2*Jr + Jp*mp*Lr**2;
A = np.array([
        [0, 0, 1, 0],
        [0, 0, 0, 1],
        [0, mp**2*(Lp/2)**2*Lr*g/Jt, -Dr*(Jp+mp*(Lp/2)**2)/Jt, -mp*(Lp/2)*Lr*Dp/Jt],
        [0, mp*g*(Lp/2)*(Jr+mp*Lr**2)/Jt, -mp*(Lp/2)*Lr*Dr/Jt, -Dp*(Jr+mp*Lr**2)/Jt]
])

B = np.array([0, 0, (Jp+mp*(Lp/2)**2)/Jt, mp*(Lp/2)*Lr/Jt])
# % Add actuator dynamics
B = kt * B / Rm;
A[2,2] = A[2,2] - kt*kt/Rm*B[2]
A[3,2] = A[3,2] - kt*kt/Rm*B[3]

In [85]:
A, B

(array([[  0.        ,   0.        ,   1.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ,   1.        ],
        [  0.        , 149.27509687, -14.92870315,  -4.91493074],
        [  0.        , 261.60910737, -14.75511358,  -8.61356429]]),
 array([ 0.        ,  0.        , 49.72753455, 49.1493074 ]))

In [86]:
k = LQR_control(A, B.reshape(4,1))
k

array([-1.        , 40.50327844, -1.60396556,  3.04438949])

In [13]:
A = np.array([
    [0., 0., 1., 0.], 
    [0., 0., 0., 1.], 
    [0., 150., -15., -5.], 
    [0., 260., -15., -9.]
])
B = np.array([0., 0., 50., 50.])

In [14]:
k = LQR_control(A, B.reshape(4,1))
k

array([-1.        , 39.87619151, -1.61166925,  3.00985469])